# Пока не доделал, но я в процессе

### Transformer for translation task



In [48]:
import pandas as pd

### Подготовка данных

Будем решать задачу машинного перевода с русского языка на английский. В качестве модели по архитектуре трансформера. Данные для обучения можно загрузить [здесь](https://www.manythings.org/anki/)

In [49]:
DATASET_PATH = "data/rus-eng/rus.txt"

In [81]:
df = pd.read_csv(DATASET_PATH, sep = "\t")
df.index.name = "sample_id"
df.head()

,eng,rus,description
sample_id,,,
0,Go.,Марш!,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Go.,Иди.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
2,Go.,Идите.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
3,Hi.,Здравствуйте.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
4,Hi.,Привет!,CC-BY 2.0 (France) Attribution: tatoeba.org #5...


Для начала нужно убрать все знаки пунктуации и столбец с описанием, а затем каждое слово превратить в числовой вектор.

In [82]:
df.drop(columns = "description", inplace = True)

In [36]:
import torch

from typing import Tuple

In [3]:
class Attention3D(torch.nn.Module):
    def __init__(self, n_features):
        super(Attention3D, self).__init__()

        self.query, self.key, self.value = [
            torch.nn.Linear(n_features, n_features) 
            for _ in range(3)
        ]

        self.softmax = torch.nn.Softmax(dim = 1)

    def forward(self, input):
        context_length, batch_size, n_features = input.shape
        output = input.clone().detach()
        
        for i in range(batch_size):
            input_2d = output[:, i: i+1, :].clone().detach().reshape(context_length, n_features)
            Q, K, V = self.query(input_2d), self.key(input_2d), self.value(input_2d)

            attention_2d = self.softmax( (Q @ K.T) / n_features**0.5) @ V
            output[:, i: i+1, :] = attention_2d.reshape(context_length, 1, n_features)
        
        return output

In [7]:
class Encoder(torch.nn.Module):
    
    def __init__(self, batch_sample: torch.Tensor) -> None:
        super(Encoder, self).__init__()
        
        context_length, batch_size, n_features = batch_sample.shape
        self.layer_norm = torch.nn.LayerNorm(normalized_shape = n_features)
        
        self.attention_layer = Attention3D(n_features)
        self.feed_forward_layer = torch.nn.Sequential(
            torch.nn.Linear(n_features, n_features * 2),
            torch.nn.Linear(n_features * 2, n_features)
        )
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        attention_output = self.attention_layer(input)
        norm_attention_output = self.layer_norm(input + attention_output)
        
        feed_forward_output = self.feed_forward_layer(norm_attention_output)
        encoder_output = self.layer_norm(norm_attention_output + feed_forward_output)
        
        return encoder_output

In [39]:
class Decoder(torch.nn.Module):
    def __init__(self, batch_sample: torch.Tensor) -> None:
        
        super(Decoder, self).__init__()
        
        context_length, batch_size, n_features = batch_sample.shape
        self.layer_norm = torch.nn.LayerNorm(normalized_shape = n_features)
        
        
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        pass

In [44]:
class Transformer(torch.nn.Module):
    
    def __init__(self, batch_sample: torch.Tensor, stack_size: int) -> None:
        super(Transformer, self).__init()
        
        self.encoders, self.decoders = [torch.nn.Sequential() for _ in range(2)]
        
        for _ in range(stack_size):
            self.encoders.append(Encoder(batch_sample))
            self.decoders.append(Decoder(batch_sample))
            
    def _exctract_encoder_key_value(self, encoder: Encoder) -> Tuple[torch.Tensor, torch.Tensor]:
        K = list(encoder.attention_layer.key.parameters())[0].data
        V = list(encoder.attention_layer.value.parameters())[0].data
        return K, V
            
    def forward(self, input: torch.Tensor) -> torch.Tensor:
        
        # 1. Forward input through encoders stack
        self.encoders(input)
        
        # 2. Now, last encoder contains needed K,V tensors
        
        # 3. Get K,V tensors for decoder cross attention sublayer
        K, V = self._exctract_encoder_key_value(self.encoders[-1])      # requires_grad ?
        
        self.decoders()